# Assignment 4: N-Gram Language Models (50 Points for both CSE 447 and CSE 517)

Author: Kabir Ahuja

Thanks to Melissa Mitchell and Khushi Khandelwal for feedback and Kavel Rao for designing the autograder.

In this project, you will implement and experiment with N-gram language models. N-gram language models are simplest versions of a language model, which make a simplifying assumption that the probability of predicting a word in a sentence only depends on the past $N$ words in the sentence. In this project you will learn:

- How to train word-level unigram and N-gram LMs on text data
- Evaluating quality of an LM by computing perplexity
- Sampling text from an N-gram LM
- Implement Laplace Smoothing
- Implement Interpolation for N-Gram Language Models

We will be working with Shakespeare Plays from Andrej Karpathy's [blog post on Recurrent neural networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/). We also recommend going through chapter 3 of [Jurafsky and Martin](https://web.stanford.edu/~jurafsky/slp3/3.pdf) on N-Gram models, especially if you are not familiar with them yet.

**Notes about the autograder for this assignment:**

*   To submit the coding part of this assignment to Gradescope, download your .ipynb notebook in .ipynb form and submit **only** this file. It must be named **CSE447_Assignment4.ipynb** in order for the autograder to work.
*   At a low-load time on Gradescope, the autograder for this assignment takes about **five minutes** to run. We recommend submitting the code part of your assignment with enough time before the deadline to allow the autograder to run, and for you to investigate/correct any issues it flags.

In [ ]:
%%bash

# get any other necessary files for this project

if [ ! -e "data-needed.txt" ]; then
  if [ ! -e "data_path_to_download_url.py" ]; then
    wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/data_path_to_download_url.py
  else
    echo "data_path_to_download_url.py script already downloaded to runtime"
  fi

  wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/assignments/NgramLanguageModels/data-needed.txt

  # download all data files needed for the student-release version of this project (i.e., no hidden test files)
  DATA_NEEDED_FILE="data-needed.txt"
  closing_slash="/"
  while IFS= read -r line; do
    line="$(echo -e "${line}" | sed -e 's/^[[:space:]]*//' -e 's/[[:space:]]*$//')";
    dirs_to_check="${line%${closing_slash}*}"
    mkdir -p $dirs_to_check
    download_url=$(python data_path_to_download_url.py "$line")
    echo $download_url;
    wget "$download_url" -O "$line"
  done < "$DATA_NEEDED_FILE"
else
  echo "data-needed.txt (and presumably therefore all necessary data files) already downloaded to runtime"
fi

if [ ! -e "other-setup-needed.sh" ]; then
  wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/assignments/NgramLanguageModels/other-setup-needed.sh
  bash other-setup-needed.sh
  rm data_path_to_download_url.py
else
  echo "other-setup-needed.sh (and presumably therefore all other necessary files) already downloaded to runtime"
fi

In [ ]:
# Load necessary packages
import os
from typing import List, Dict
import random
import numpy as np

# Setup nltk
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# Set data directory. Important: DO NOT CHANGE THIS. AUTOGRADER WILL FAIL ON YOUR SUBMISSION OTHERWISE
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, "data")

In [ ]:
# Helper functions for sample test cases

def evaluate_test_case(input, output, expected_output, output_str = "Output", atol=1e-4) -> Dict:

    if input is not None:
        print("Input:\n", input)
    print(f"{output_str}:\n", output)
    print(f"Expected {output_str}:\n", expected_output)

    match = (
        output == expected_output
        if type(output) == str
        else np.allclose(output, expected_output, atol=atol)
    )

    if match:
        print("Test case passed! :)")

    else:
        print("Test case failed! :(")

    print("\n" + "="*50 + "\n")


def evaluate_list_test_case(input: List, output: List, expected_output: List) -> Dict:

    print("Input:\n", input)
    print("Output:\n", output)
    print("Expected output:\n", expected_output)
    if output == expected_output:
        print("Test case passed! :)")

    else:
        print("Test case failed! :(")

    print("\n" + "="*50 + "\n")

## Part 1: Word-level unigram language models (11 points)

We will start by considering word-level language models i.e. language models where the smallest unit (or a unigram) that can be predicted by the model is a word. In part 1, we will implement unigram language models, which constitutes the simplest variant of N-gram models -- simply learn the distribution of each unigram (here a word) in the corpus. Recall from the lectures, that for a text sequence with unigrams $w_1, w_2, \cdots, w_n$, unigram language models, the probability of the sequence is given as:

$$P(w_1, w_2, \cdots, w_n) =P(w_1)P(w_2)\cdots P(w_n)$$

where $P(w_i)$ is simply the frequency of the word $w_i$ in the training corpus.

For this part we will work with the Shakespeare dataset. Let's start by loading the datasets

In [ ]:
with open(f"{data_dir}/shakespeare/shakespeare_train.txt") as f:
    train_data = f.read().split("\n")

with open(f"{data_dir}/shakespeare/shakespeare_dev.txt") as f:
    dev_data = f.read().split("\n")

Below we print first 10 sentences from the training data.

In [ ]:
print("\n".join(train_data[:10]))

### Exercise 1.0 Text Processing

Before start training our models, let's perform some basic preprocessing. For this exercise, we only want to put an \<eos\> tag at the end of each sentence in the training and dev sets. Implement `add_eos` function below that does that.

In [ ]:
def add_eos(data: List[str]) -> List[str]:
    """
    Adds an <eos> token to the end of each line in the data.

    Inputs:
    - data: a list of strings where each string is a line of text

    Returns:
    - a list of strings where each string is a line of text with <eos> token appended
    """

    raise NotImplementedError


In [ ]:
def test_add_eos():
    print("Running Sample Test Case 1")
    data = ["hello!", "world!"]

    evaluate_list_test_case(data, add_eos(data), ["hello! <eos>", "world! <eos>"])

    print("Running Sample Test Case 2")
    data = [
        "Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ." ]

    expected_output = [
        "Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice . <eos>",
        "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs . <eos>",
        "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point . <eos>"]
    evaluate_list_test_case(data, add_eos(data), expected_output)

test_add_eos()

In [ ]:
train_data_processed = add_eos(train_data)
dev_data_processed = add_eos(dev_data)

### Exercise 1.1: Training (word-level) Unigram Language Model (2 Points)

Training a unigram model simply corresponds to calculating frequencies of each word in the corpus, i.e.

$$p(w_i) = \frac{C(w_i)}{n}$$

where $C(w_i)$ is the count of word $w_i$ in the training data and $n$ is the total number of words in the training dataset.

Implement the `train_word_unigram` function below

In [ ]:
def train_word_unigram(train_data: List[str]) -> Dict[str, float]:

    """
    Trains a word-level unigram language model.

    Inputs:
        - train_data: List[str], list of sentences in training data

    Outputs:
        - Dict[str, float], a dictionary mapping words to their unigram probabilities

    """

    unigram_probs = {}


    raise NotImplementedError

    return unigram_probs

In [ ]:
unigram_probs = train_word_unigram(train_data_processed)

In [ ]:
# Sample test cases
def test_train_word_unigram(unigram_probs):

    print("Running Sample Test Case 1: Check if the number of unique words is correct")
    evaluate_test_case(None, len(unigram_probs), 12610, output_str="Number of unique words")

    print("Running Sample Test Case 2: Check if the probability of word \"thou\" is correct")
    evaluate_test_case("thou", unigram_probs["thou"], 0.004559649641510829)

    print("Running Sample Test Case 3: Check if the probability of word \"love\" is correct")
    evaluate_test_case("love", unigram_probs["love"], 0.0015984182127282134)

    print("Running Sample Test Case 4: Check if the probability of word \"Richard\" is correct")
    evaluate_test_case("Richard", unigram_probs["Richard"], 0.0005035479340675586)

    print("Running Sample Test Case 5: Check if the probability of word \"pytorch\" is correct")
    evaluate_test_case("pytorch", unigram_probs.get("pytorch", 0), 0)

    print("Running Sample Test Case 6: Check if the probability of word \"richard\" is correct")
    evaluate_test_case("richard", unigram_probs.get("richard", 0), 0)


test_train_word_unigram(unigram_probs)

### Exercise 1.2: Evaluating (word-level) Unigram Language Models using Perplexity (2 Points)

Now that we have trained our first (albeit very basic) language model, our next job is to evaluate how good of a job it does in modeling the training text as well as generalizing on the unseen text. The most commonly used metric for evaluating the quality of a language model is Perplexity. Recall from the lecture, perplexity of a language model on a test dataset measures the (inverse) probability assigned by the language model to the test dataset normalized by the number of words (or tokens). Lower the perplexity the higher probability the model assigns to the text in the test dataset and hence better quality.

$$\text{perplexity}(W) = P(w_1w_2\cdots w_n)^{\frac{-1}{n}} = \sqrt[n]{\frac{1}{P(w_1w_2\cdots w_n)}}$$

where $W$ is a test set with $n$ words $w_1w_2\cdots w_n$

It is useful to do perplexity calculation in log space to avoid numerical issues

$$\text{perplexity}(W) = \exp\bigl(-\frac{\log{P(w_1w_2\cdots w_n)}}{n}\bigr)$$

When we have multiple sentences in the corpus and we assume sentences to be independent, we can write:

$$\text{perplexity}(W) = \exp\bigl(-\frac{ \sum_{S \in W}  \log{P(s_1s_2\cdots s_{n_s})}}{n}\bigr)$$

where $S$ is a sentence in the corpus $W$ with words $s_1 s_2 \cdots s_{n_s}$ and $n_s$ is the number of words in $S$.

Note that assuming sentences to be independent is something that is not actually true in practice. However, since N-gram language models are already limited in their context, it is not the greatest loss to remove dependencies between sentences. In the future homeworks we will be dropping this assumption as we build more powerful models.


In [ ]:
def eval_ppl_word_unigram(eval_data: List[str], unigram_probs: Dict[str, float]) -> float:

    """
    Evaluates the perplexity of a word-level unigram language model on the dev set.

    Inputs:
        - dev_data: List[str], list of sentences in the evaluation data
        - unigram_probs: Dict[str, float], a dictionary mapping words to their unigram probabilities

    Outputs:
        - float, the perplexity of the unigram language model on the evaluation set

    Note 1: It is useful to do the calculations in log space and convert the final answer to original space to
    avoid numerical issues .
    Note 2: Assign 0 probability to words that are not in the unigram_probs dictionary, since those words were not seen during training.
    """

    perplexity = 0


    raise NotImplementedError

    return perplexity

First let's compute the perplexity of training data

In [ ]:
train_ppl = eval_ppl_word_unigram(train_data_processed, unigram_probs)
print(train_ppl)

You should expect a perplexity around 575 on the training data. Now let's evaluate on dev data.

In [ ]:
dev_ppl = eval_ppl_word_unigram(dev_data_processed, unigram_probs)
print(dev_ppl)

You should see a RuntimeWarning: divide by zero and a perplexity of infinity in this case. Why did this happen? Because we have some words in the dev dataset, which were never seen during training. The unigram model assigns a zero probability to those, which result in an infinite perplexity (inverse probability).

### Exercise 1.3: Handling Unknown Words (2 Points)

How do we deal with such situations? An easy solution is to introduce an unknown word token, e.g. \<unk\>. Before training, we replace the words which occur less than a threshold number of times with an \<unk\> token. E.g., if a word occurs less than 3 times in the dataset, we replace it with the \<unk\> token. At test time, when evaluating the perplexity if we see a word, which was unseen during training i.e. has a unigram frequency of 0, we assign that word the probability of \<unk\> token.

Implement the `replace_rare_words_with_unks` function below which replaces words which occur less than `unk_thresh` number of times by \<unk\> token. Also reimplement `eval_ppl_word_unigram` to handle unseen words. You might find the [`Counter` class from the `collections` module useful](https://docs.python.org/3/library/collections.html#counter-objects).

In [ ]:
def replace_rare_words_with_unks(train_data: List[str], unk_thresh: int) -> List[str]:
    """
    Replaces words that occur less than unk_thresh times in the training data with "<unk>" token.

    Inputs:
        - train_data: List[str], list of sentences in the training data
        - unk_thresh: int, the threshold on the number of occurrences of a word to be considered known (less than considered <unk>)

    Outputs:
        - List[str], the training text with rare words replaced by "<unk>" token
    """

    train_data_unked = []


    raise NotImplementedError

    return train_data_unked

In [ ]:
# Sample test cases
def test_train_word_unigram_wth_unks():

    print("# Testing for Threshold 3\n\n")
    train_data_wth_unks = replace_rare_words_with_unks(train_data_processed, 3)
    unigram_probs_wth_unks = train_word_unigram(train_data_wth_unks)
    print("Sample Test Case 1: Check if the number of unique words is correct")
    evaluate_test_case(None, len(unigram_probs_wth_unks), 4620, output_str="Number of unique words")

    print("Sample Test Case 2: Check if the probability of word \"<unk>\" is correct")
    evaluate_test_case("<unk>", unigram_probs_wth_unks["<unk>"], 0.045185342597383396)

    print("# Testing for threshold 5\n\n")
    train_data_wth_unks = replace_rare_words_with_unks(train_data_processed, 5)
    unigram_probs_wth_unks = train_word_unigram(train_data_wth_unks)
    print("Sample Test Case 3: Check if the number of unique words is correct")
    evaluate_test_case(None, len(unigram_probs_wth_unks), 3088, output_str="Number of unique words")

    print("Sample Test Case 4: Check if the probability of word \"<unk>\" is correct")
    evaluate_test_case("<unk>", unigram_probs_wth_unks["<unk>"], 0.06910155961268387)

test_train_word_unigram_wth_unks()

In [ ]:
# We will remove words that occur less than 3 times
train_data_wth_unks = replace_rare_words_with_unks(train_data_processed, 3)
unigram_probs_wth_unks = train_word_unigram(train_data_wth_unks)

We recommend you to check the unigram probabilities before and after replacing rare words with \<unk\> token to verify your implementation.

In [ ]:
def eval_ppl_word_unigram_with_unks(eval_data: List[str], unigram_probs_wth_unks: Dict[str, float]) -> float:

    """
    Evaluates the perplexity of a word-level unigram language model on the dev set. For unseen words, uses the <unk> token probability.

    Inputs:
        - eval_data: string, List of sentences in eval data
        - unigram_probs_wth_unks: Dict[str, float], a dictionary mapping words to their unigram probabilities, including <unk> token

    Outputs:
        - float, the perplexity of the unigram language model on the evaluation set

    """

    perplexity = 0


    raise NotImplementedError

    return perplexity

Let's compute the train and dev perplexity now

In [ ]:
train_ppl = eval_ppl_word_unigram_with_unks(train_data_wth_unks, unigram_probs_wth_unks)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = eval_ppl_word_unigram_with_unks(dev_data_processed, unigram_probs_wth_unks)
print(f"Dev Perplexity: {dev_ppl}")

You should observe a train perplexity around 384 and dev perplexity around 282.

### Exercise 1.4: Sampling from Unigram Language Model (2 Points)

Now that we have trained and evaluated our unigram LM, we are ready to generate some text from it. To sample text from an N-gram language model given prefix words $w_1, w_2, \cdots, w_n$, we sequentially sample next tokens from the N-gram probability distribution given the previous words, i.e.,

$$w_{n+1} \sim P(w_{n+1} | w_{1}, \cdots, w_{n} )$$

For a unigram language model, since $P(w_1, \dots, w_n) = P(w_1)\cdots P(w_n)$ i.e. all words all distributed independently and the next token is sampled independent of previous tokens, the above equation simplifies to:

$$w_{n+1} \sim P(w_{n+1})$$

You will now implement the function `sample_from_word_unigram` below. You might find the [Numpy's random module (np.random)](https://numpy.org/doc/stable/reference/random/index.html) useful.

In [ ]:
def sample_from_word_unigram(unigram_probs: Dict[str, float], max_words: int, prefix: str = "") -> str:

    """
    Samples sequence of words from a unigram language model.
    Terminate sampling when either max_words is reached or when <eos> token is sampled.

    Inputs:
        - unigram_probs: Dict[str, float], a dictionary mapping words to their unigram probabilities
        - n_words: int, the number of words to sample
        - prefix: str, a prefix to start the sampling from. Can have multiple words separated by spaces.

    Outputs:
        - str: sampled text i.e. string of sampled words separated by spaces along with the prefix

    # Note: Please use np.random.choice to sample from the unigram_probs dictionary.
    """
    sampled_string = ""

    raise NotImplementedError

    return sampled_string

In [ ]:
# Sample test cases
def test_sample_from_word_unigram():

    np.random.seed(0)
    sampled_string = sample_from_word_unigram(unigram_probs_wth_unks, 10, "The king")
    print("Running Test Case 1: Check if the sampled string starts with the prefix")
    evaluate_test_case(None, sampled_string.startswith("The king"), True, output_str="Sampled string starts with the prefix")

    print("Running Test Case 2: Check if the sampled string has either 10 generated words or ends with <eos> token")
    print(f"Generated string: {sampled_string}")
    print(f"Number of generated words: {len(sampled_string.split()) - 2}")
    print(f"Does the generated string end with <eos> token: {'<eos>' in sampled_string}")
    if len(sampled_string.split()) - 2 == 10 or (
        len(sampled_string.split()) - 2 < 10 and "<eos>" in sampled_string
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")
    print("\n" + "="*50 + "\n")

    print("Running Test Case 3: Check if the probability of generating <unk> token is correct")

    sampled_strings = [
        sample_from_word_unigram(unigram_probs_wth_unks, 1, "")
        for _ in range(1000)
    ]
    sampled_string = " ".join(sampled_strings)
    num_unks = sampled_string.count("<unk>")
    unk_gen_prob = num_unks / len(sampled_string.split())
    evaluate_test_case(None, unk_gen_prob, unigram_probs_wth_unks["<unk>"], output_str="Probability of generating <unk> token", atol=1e-2)

test_sample_from_word_unigram()

Note that due to the randomness in sampling, it is difficult to automatically test this function. However, you can use the technique we use in sample test case 3 by repeatedly sampling from the unigram model and checking the frequency of different words in the generated text and checking if they are close to the unigram probabilities.

Let's sample some text from the unigram model

In [ ]:
for _ in range(10):
    sampled_string = sample_from_word_unigram(unigram_probs_wth_unks, 20)
    print(sampled_string)

You should see that the generated that doesn't make a whole lot of sense, which is natural since we are using a unigram model that doesn't account for the context at all, and essentially samples the most common tokens in its generations. We will now move to build language models that do not have this problem, i.e., they take into account the context (albeit to different degrees) for modeling the distribution of words (or tokens) in the text.

### Write-Up Question 1: Effect of \<unk\> tokens on perplexity and generation quality (2 Points)

What effect do you think the \<unk\> tokens will have on the perplexity of the model? Try out different thresholds for replacing rare words with \<unk\> token and report the perplexity on the training and dev set. What do you observe? Does the perplexity decrease with increasing threshold? Do you think that improves generation quality? (Your answer to this question should go in your separate write-up PDF.)

**What to submit:**

- A Table with the perplexity of the unigram model on the training and dev set for different thresholds of replacing rare words with \<unk\> token. You can choose the thresholds as 1, 3, 5,  7, 9, 10.

- Example generations at each threshold.

- 3-4 lines maximum discussing the trends you observe in the perplexity and generation quality with increasing threshold.

In [ ]:

# YOUR CODE HERE


### Write-Up Question 2: Alterative to Random Sampling (1 Point)

An alternate algorithm to generate text from a language model is greedy decoding, i.e., where we generate the most likely token at each step of decoding, i.e.

$$ w_{k+1} = \texttt{argmax}_{w} P(w | w_{1}, \cdots, w_{k}) $$

Can you explain why or why not that will be a good idea for unigram language models? Explain in no more in 3 lines. (Your answer to this question should go in your separate write-up PDF.)

## Part 2: N(>1)-Gram Word-Level Language Models (12 Points)

We will now implement much more sophisticated language models, which make use of the surrounding text to model the distribution of text. Recall from the lectures for an $N$-gram language model with $n > 1$, the distribution of a sequence of tokens $w_1, w_2, \cdots, w_n$ is given as:

$$P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-N-1}, \cdots, w_{k-1})$$

E.g., for a bigram model i.e. $N = 2$, the expression becomes:

$$P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-1})$$

i.e. the distribution of a token depends solely on the tokens preceding it.

### Write-Up Question 3 (2 Points)

Can you show why for an N-gram language model the following expression holds?

$$P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-N-1}, \cdots, w_{k-1})$$

Lay down the assumption that is required to derive this expression and show all the steps in your derivation. (Your answer to this question should go in your separate write-up PDF.)


In [ ]:
# Before we start, lets set training data to be the one with <unk> tokens and dev data to be the one with <eos> tokens in the end

train_data = train_data_wth_unks
dev_data = dev_data_processed

### Exercise 2.1: Text Processing

A careful reader must have noted that the above expression: $P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-1})$ has a problem. On the right hand side, when $k = 1$, this will result in the term $P(w_1 | w_0)$, but there is no $w_0$ in the sequence. Similarly, for a trigram language model we will have terms, $P(w_1 | w_{-1}, w_{0})$ and $P(w_2 | w_0, w_1)$ with conditionals on words that are not part of the sequence. To handle this issue, we add $N-1$ start of sequence tokens, e.g. \<sos\> to the beginning of the sequence.

Implement `process_text_for_Ngram` function below that adds $N-1$ \<sos\> tokens to beginning of each sentence in the dataset.

In [ ]:
def process_text_for_Ngram(sents: List[str], N: int = 2) -> List[str]:

    """
    Adds N-1 <sos> tokens to the start of every sentence in the text.

    Inputs:
        - sents: List[str], List of sentences
        - N: int, the N in N-gram

    Outputs:
        - List[str], the processed text
    """
    processed_sents = None


    raise NotImplementedError

    return processed_sents

In [ ]:
# Sample test cases

def test_process_text_for_Ngram():

    sents = ["Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
    "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
    "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ." ]

    print("Running Sample Test Case 1 with N=1")
    unigram_processed_text = process_text_for_Ngram(sents, 1)
    excepted_output = [
        "Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, unigram_processed_text, excepted_output)

    print("Running Sample Test Case 2 with N=2")
    bigram_processed_text = process_text_for_Ngram(sents, 2)
    excepted_output = [
        "<sos> Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "<sos> At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "<sos> The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, bigram_processed_text, excepted_output)

    print("Running Sample Test Case 3 with N=3")
    trigram_processed_text = process_text_for_Ngram(sents, 3)
    excepted_output = [
        "<sos> <sos> Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "<sos> <sos> At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "<sos> <sos> The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, trigram_processed_text, excepted_output)

    print("Running Sample Test Case 4 with N=4")
    fourgram_processed_text = process_text_for_Ngram(sents, 4)
    excepted_output = [
        "<sos> <sos> <sos> Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "<sos> <sos> <sos> At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "<sos> <sos> <sos> The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, fourgram_processed_text, excepted_output)

test_process_text_for_Ngram()

### Exercise 2.2: Implementing N-gram language models (10 Points)

The heart of implementing an N-gram language model is to estimate the conditional distributions $P(w_n \mid w_{n-N-1}, \cdots, w_{n-1})$. Recall from the lectures that the conditional distributions can be estimated as:

$$P(w_n \mid w_{n-N-1}, \cdots, w_{n-1}) = \frac{C(w_{n-N-1} \cdots w_{n-1} w_{n})}{\sum_{w \in \mathcal{W}}{C(w_{n-N-1} \cdots w_{n-1} w)}} = \frac{C(w_{n-N-1} \cdots w_{n-1} w_{n})}{{C(w_{n-N-1} \cdots w_{n-1})}}$$

where $C(w_{n-N-1} \cdots w_{n-1} w)$ is the number of times the token sequence $w_{n-N-1} \cdots w_{n-1} w$ appears in the corpus, and $\mathcal{W}$ is the vocabulary of the N-gram model.

You will now implement an N-gram language model from scratch. Note that a full implementation involves a `fit` function, which computes the N-gram counts $C(w_{n-N-1} \cdots w_{n-1} w)$ needed to compute the conditional distributions; `eval_perplexity` function, which evaluates the perplexity of the language model on a test set; and a `sample_text` function which generates the text from the LM. Implement the class `WordNGramLM` below with these functions.This time we leave all the implementation details for you to decide. We just provide a boiler plate code, with the expected input output for the three functions in the class. You might need to implement additional functions for your implementation of the three functions.

**Note 1**: If implementing a general NGram Model from scratch seems too daunting, we recommend starting implementing just the Bigram LM and checking if you are able to pass BigramLM Test cases. From there you can work on generalizing your solution.

**Note 2**: Efficiency of your code will be important here, especially for `eval_perplexity` and `sample_text` functions. A naive implementation of these functions will result in a time complexity of $\mathcal{O}(nV)$ for `eval_perplexity` and $\mathcal{O}(TV)$ for `sample_text`, where $n$ is the number of words in evaluation dataset, T is the length of sampled text, and $V$ is the size of vocabulary. By caching certain quantities during training, you should be able to write implementations with time complexities $\mathcal{O}(n + V)$ and $\mathcal{O}(T + V)$ for the two functions. We will give only half credit for the naive implementation.

**Note 3**: We will provide sample test cases only to test `eval_perplexity` and `sample_text` functions and not for the `fit` function. Both of the former functions rely on the latter to be implemented correctly, hence use correctness of `eval_perplexity` and `sample_text` to check the correctness of your `fit` function.

In [ ]:
class WordNGramLM:

    def __init__(self, N: int):
        self.N = N

    def fit(self, train_data: List[str]):

        """
        Trains an N-gram language model.

        Inputs:
            - train_data: str, sentences in the training data

        """


        raise NotImplementedError

    def eval_perplexity(self, eval_data: List[str]) -> float:

        """
        Evaluates the perplexity of the N-gram language model on the eval set.

        Input:
            - eval_data: List[str], the evaluation text

        Output:
            - float, the perplexity of the model on the evaluation set

        Note : For words that are not in the vocabulary, replace them with the <unk> token.
        Note : Don't count the <sos> tokens in your number of total tokens in order to match expected perplexities.

        """


        raise NotImplementedError

    def sample_text(self, prefix: str = "<sos>", max_words: int = 100) -> float:

        """
        Samples text from the N-gram language model.
        Terminate sampling when either max_words is reached or when <eos> token is sampled.
        Inputs:
            - prefix: str, the prefix to start the sampling from. Can also be multiple words separated by spaces.
            - max_words: int, the maximum number of words to sample

        Outputs:
            - str, the sampled text

        Note: Please use np.random.choice for sampling next words
        """


        raise NotImplementedError

    # Extra utility functions that you think will be useful can go below

    # YOUR CODE HERE


In [ ]:
# <NO_AUTOGRADE>

As a sanity check, check if your code returns same perplexities when N=1 as your earlier unigram model perplexities

In [ ]:
unigram_lm = WordNGramLM(1)
unigram_lm.fit(train_data)

train_ppl = unigram_lm.eval_perplexity(train_data)
dev_ppl = unigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Unigram model (Class Implementation): {train_ppl}")
print(f"Dev Perplexity for Unigram model (Class Implementation): {dev_ppl}")

train_ppl_old = eval_ppl_word_unigram_with_unks(train_data_wth_unks, unigram_probs_wth_unks)
dev_ppl_old = eval_ppl_word_unigram_with_unks(dev_data_processed, unigram_probs_wth_unks)

print(f"Train Perplexity for Unigram model (Original Implementation): {train_ppl_old}")
print(f"Dev Perplexity for Unigram model (Original Implementation): {dev_ppl_old}")

if np.allclose(train_ppl, train_ppl_old, atol = 1e-4) and np.allclose(dev_ppl, dev_ppl_old, atol = 1e-4):
    print("Unigram model perplexities match! :)")

else:
    print("Unigram model perplexities do not match! :(")

# assert np.allclose(train_ppl, train_ppl_old, atol = 1e-4)
# assert np.allclose(dev_ppl, dev_ppl_old, atol = 1e-4)
# print("Unigram model perplexities match!")


Test implementation of `eval_perplexity` for bigram model

In [ ]:
# Test implementation of `eval_perplexity` for bigram model
bigram_lm = WordNGramLM(2)
bigram_lm.fit(train_data)
train_ppl = bigram_lm.eval_perplexity(train_data)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = bigram_lm.eval_perplexity(dev_data)
print(f"Dev Perplexity: {dev_ppl}")

You should see a train perplexity of roughly 42 and dev perplexity infinite, we will soon see how to deal with that.

Test implementation of `sample_text` for bigram model

In [ ]:
def test_sample_text_ngram():

    print("Testing for Trigram model")

    random.seed(42)
    np.random.seed(42)
    trigram_lm = WordNGramLM(3)
    trigram_lm.fit(train_data)
    sampled_text = trigram_lm.sample_text("<sos> <sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos> <sos>")
    evaluate_test_case(None, sampled_text.startswith("<sos> <sos>"), True, output_str="Sampled text starts with <sos> <sos>")

    print("Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>")
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 2}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 2 == 50 or (
        len(sampled_text.split()) - 2 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 3: Check if the probability of generating II is greater than III when prefix is KING RICHARD")
    sampled_texts = [
        trigram_lm.sample_text("KING RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("KING RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("KING RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)

    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 4: Check if the probability of generating II given KING RICHARD are close to the expected values")
    evaluate_test_case("King Richard II", gen_prob_richard_2, 0.4051, output_str="Probability of generating Richard II", atol=1e-2)

    print("Test Case 5: Check if the probability of generating III given KING RICHARD are close to the expected values")
    evaluate_test_case("King Richard III", gen_prob_richard_3, 0.5949, output_str="Probability of generating Richard III", atol=1e-2)

    print("Testing for 4-gram model")
    random.seed(42)
    np.random.seed(42)
    fourgram_lm = WordNGramLM(4)
    fourgram_lm.fit(train_data)
    # sampled_text = fourgram_lm.sample_text("<sos> <sos>", max_words=50)

    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos>", max_words=50)

    print("Test Case 6: Check if the sampled text starts with <sos> <sos> <sos>")
    evaluate_test_case(None, sampled_text.startswith("<sos> <sos> <sos>"), True, output_str="Sampled text starts with <sos> <sos> <sos>")

    print("Test Case 7: Check if the number of generated words is either 50 or less than 50 and ends with <eos>")
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 3}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 3 == 50 or (
        len(sampled_text.split()) - 3 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")

    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 8: Check if the probability of generating II is greater than III when prefix is <sos> KING RICHARD")
    sampled_texts = [
        fourgram_lm.sample_text("<sos> <sos> KING RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)

    num_richard_3s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)

    gen_prob_rich2 = num_richard_2s / len(sampled_texts)
    gen_prob_rich3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_rich2}")
    print(f"Probability of generating Richard III: {gen_prob_rich3}")
    if gen_prob_rich2 < gen_prob_rich3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 9: Check if the probability of generating II given <sos> KING RICHARD are close to the expected values")
    evaluate_test_case("<sos> King Richard II", gen_prob_rich2, 0.4044, output_str="Probability of generating Richard II", atol=1e-2)

    print("Test Case 10: Check if the probability of generating III given <sos> KING RICHARD are close to the expected values")
    evaluate_test_case("<sos> King Richard III", gen_prob_rich3, 0.5956, output_str="Probability of generating Richard III", atol=1e-2)

test_sample_text_ngram()

In [ ]:
def test_sample_text_bigram_model():
    bigram_lm = WordNGramLM(2)
    bigram_lm.fit(train_data)

    random.seed(42)
    np.random.seed(42)
    sampled_text = bigram_lm.sample_text("<sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos>")
    evaluate_test_case(None, sampled_text.startswith("<sos>"), True, output_str="Sampled text starts with <sos>")

    print("Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>")
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 1}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 1 == 50 or (
        len(sampled_text.split()) < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 3: Check if the probability of generating II is greater than III when prefix is RICHARD")
    sampled_texts = [
        bigram_lm.sample_text("RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)
    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print(
        "Test Case 4: Check if the probability of generating II given RICHARD  are close to the expected values"
    )
    evaluate_test_case("Richard II", gen_prob_richard_2, 0.35251798561151076, output_str="Probability of generating Richard II", atol=1e-2)

    print(
        "Test Case 5: Check if the probability of generating III given RICHARD are close to the expected values"
    )
    evaluate_test_case("Richard III", gen_prob_richard_3, 0.49640287769784175, output_str="Probability of generating Richard III", atol=1e-2)

test_sample_text_bigram_model()

In [ ]:
for _ in range(20):
    sampled_text = bigram_lm.sample_text("<sos> KING", max_words=50)
    print(sampled_text)

The generations should look much better now, way more coherent compared to the unigram model! At least on the surface level it seems to capture the style of Shakespeare's writing.

Test implementation of `eval_perplexity` for trigram, 4-gram, and 5-gram models

In [ ]:
trigram_lm = WordNGramLM(3)
trigram_lm.fit(train_data)

train_ppl = trigram_lm.eval_perplexity(train_data)
dev_ppl = trigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model: {train_ppl}")
print(f"Dev Perplexity for Trigram model: {dev_ppl}")

print("\n\n")

fourgram_lm = WordNGramLM(4)
fourgram_lm.fit(train_data)
train_ppl = fourgram_lm.eval_perplexity(train_data)
dev_ppl = fourgram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 4-gram model: {train_ppl}")
print(f"Dev Perplexity for 4-gram model: {dev_ppl}")

print("\n\n")

fivegram_lm = WordNGramLM(5)
fivegram_lm.fit(train_data)
train_ppl = fivegram_lm.eval_perplexity(train_data)
dev_ppl = fivegram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 5-gram model: {train_ppl}")
print(f"Dev Perplexity for 5-gram model: {dev_ppl}")

You should see train perplexities approximately 5.9, 2.1, and 1.6 for trigram, 4-gram, and 5-gram LMs. For dev data the dev perplexity should be infinity for all the three.

Test implementation of `sample_text` for trigram and 4-gram LMs.

In [ ]:
np.random.seed(42)
print("Generations from Trigram model")
for _ in range(20):
    sampled_text = trigram_lm.sample_text("<sos> <sos> KING", max_words=50)
    print(sampled_text)

In [ ]:
np.random.seed(42)
random.seed(42)
print("Generations from 4-gram model")
for _ in range(20):
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

In [ ]:
np.random.seed(42)
random.seed(42)
print("Generations from 5-gram model")
for _ in range(20):
    sampled_text = fivegram_lm.sample_text("<sos> <sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

In [ ]:
# </NO_AUTOGRADE>

You should see that the generation quality is now so much better. However, if you look closely (this is specially true for 4-gram and 5-gram models) that some of the sentences are directly lifted from the training data. Which makes sense, as we will increase the order of the N-gram LM, so does we increase its capacity and hence more the ability to memorize its training data.

## Part 3: Smoothing and Interpolation (27 Points)

The issue with using N-gram language models is that any finite training corpus is bound to miss some N-grams that appear in the test set. The models hence assign zero probability to such N-grams, leading to probability of the entire test set to be zero and hence infinite perplexity values that we observed in the previous exercise.

The standard way to deal with zero-probability N-gram tokens is to use smoothing algorithms. Smoothing algorithms shave off a bit of probability mass from some more frequent events and give it to unseen events. Smoothing algorithms for N-gram language models is a well studied area of research with numerous algorithms. For this exercise, we will focus on Laplace Smoothing and Interpolation.

**Hint:** Try to use the inheritance from `WordNGramLM` to simplify your implementations for this part. You can use `super().method(arguments)` to call a method from the super class.

### Exercise 3.1 Laplace and Add-Lambda Smoothing (10 Points)

Perhaps the simplest smoothing algorithm that exists is Laplace smoothing. It merely adds one to count of each N-gram, so that there is no zero-probability N-gram in the test data. For a bigram model, the expression for Laplace-smoothened distribution is given by:

$$P_{\text{Laplace}}(w_n \mid w_{n-1}) = \frac{C(w_{n-1}w_{n}) + 1}{\sum_{w \in \mathcal{W}} (C(w_n w) + 1) } = \frac{C(w_{n-1}w_{n}) + 1}{ C(w_{n-1}) + V }$$

We can similarly write expressions for other N-gram models.

Laplace smoothing is also called "Add-one" smoothing. A generalization of Laplace Smoothing is "Add-k" smoothing with  with $\lambda < 1$, where we move a bit less of the probability mass from seen to unseen N-grams. The expression for Add-k smoothened distribution for bigram LM is given by:

$$P_{\text{Add-k}}(w_n \mid w_{n -1}) = \frac{C(w_{n-1}w_{n}) + \lambda}{\sum_{w \in \mathcal{W}} (C(w_n w) + \lambda) } = \frac{C(w_{n-1}w_{n}) + \lambda}{ C(w_{n-1}) + \lambda V }$$

We will use the variable `k` to represent Lambda in the code.

For this exercise, we ask you to implement `WordNGramLMWithAddKSmoothing` class. You need to follow the same code structure as `WordNGramLM` class, with only difference being in calculation of the conditional distributions.

Note: It is no longer possible to implement an O(T + V) time complexity solution for `sample_text` function, hence we will accept O(TV) implementations here. Your `eval_perplexity` should still be as efficient as in the `WordNGramLM`


In [ ]:
class WordNGramLMWithAddKSmoothing(WordNGramLM):
    """
    Remember you can use the inheritance from WordNGramLM in your implementation!
    """

    def __init__(self, N: int, k: int = 1):
        super().__init__(N)
        self.k = k

    def fit(self, train_data: List[str]):
        """
        Trains an N-gram language model with Add-k smoothing.

        Inputs:
            - train_data: str, sentences in the training data

        """


        raise NotImplementedError

    def eval_perplexity(self, eval_data: List[str]) -> float:
        """
        Evaluates the perplexity of the N-gram language model with Add-k smoothing on the eval set.

        Input:
            - eval_data: List[str], the evaluation text

        Output:
            - float, the perplexity of the model on the evaluation set

        Note : For tokens that are not in the vocabulary, replace them with the <unk> token.

        """


        raise NotImplementedError

    def sample_text(
        self, prefix: str = "<sos>", max_words: int = 100,
    ) -> float:
        """
        Samples text from the N-gram language model.

        Inputs:
            - prefix: str, the prefix to start the sampling from. Can also be multiple words separated by spaces.
            - max_words: int, the maximum number of words to sample

        Outputs:
            - str, the sampled text

        Note: Please use np.random.choice for sampling next words
        """


        raise NotImplementedError

    # Extra utility functions that you think will be useful can go below

    # YOUR CODE HERE


In [ ]:
# <NO_AUTOGRADE>

Test implementation of `eval_perplexity` for bigram model with Laplace smoothing

In [ ]:
# Test implementation of `eval_perplexity` for bigram model with Laplace smoothing
bigram_lm = WordNGramLMWithAddKSmoothing(2, k = 1)
bigram_lm.fit(train_data)
train_ppl = bigram_lm.eval_perplexity(train_data)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = bigram_lm.eval_perplexity(dev_data)
print(f"Dev Perplexity: {dev_ppl}")

You should get a train perplexity around 404 and dev perplexity around 390. Notice how the dev perplexity is not $\infty$ anymore! Though it comes at the cost of an increase in perplexity on training data, which is natural since the model now cut offs some probability mass from training N-grams and distribute it to the unseen ones.

Test implementation of `eval_perplexity` for bigram model with Add-k smoothing (k = 0.01)

In [ ]:
# Test implementation of `eval_perplexity` for bigram model with Add-k smoothing
bigram_lm = WordNGramLMWithAddKSmoothing(2, k=0.01)
bigram_lm.fit(train_data)
train_ppl = bigram_lm.eval_perplexity(train_data)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = bigram_lm.eval_perplexity(dev_data)
print(f"Dev Perplexity: {dev_ppl}")

You should get a train perplexity around 61 and a dev perplexity of roughly 148. Notice how we do much better on train perplexity when k is smaller, since now we re-distribute much less of the mass from the training N-grams. Luckily, in this case it turns out it improves the dev perplexity too.

Test implementation of `sample_text` for bigram model with Laplace Smoothing

In [ ]:
def test_sample_text_bigram_laplace_model():
    bigram_lm = WordNGramLMWithAddKSmoothing(2, k = 1)
    bigram_lm.fit(train_data_wth_unks)

    random.seed(42)
    np.random.seed(42)
    sampled_text = bigram_lm.sample_text("<sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos>")
    evaluate_test_case(
        None,
        sampled_text.startswith("<sos>"),
        True,
        output_str="Sampled text starts with <sos>",
    )

    print(
        "Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>"
    )
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 1}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 1 == 50 or (
        len(sampled_text.split()) < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 3: Check if the probability of generating II is greater than III when prefix is RICHARD"
    )
    sampled_texts = [
        bigram_lm.sample_text("RICHARD", max_words=1) for _ in range(1000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)
    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 4: Check if the probability of generating II given RICHARD  are close to the expected values"
    )
    evaluate_test_case(
        "Richard II",
        gen_prob_richard_2,
        0.014,
        output_str="Probability of generating Richard II",
        atol=1e-3,
    )

    print(
        "Test Case 5: Check if the probability of generating III given RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "Richard III",
        gen_prob_richard_3,
        0.034,
        output_str="Probability of generating Richard III",
        atol=1e-3,
    )

test_sample_text_bigram_laplace_model()

In [ ]:
random.seed(42)
np.random.seed(42)
bigram_lm = WordNGramLMWithAddKSmoothing(2, k = 1)
bigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = bigram_lm.sample_text("<sos> KING", max_words=50)
    print(sampled_text)

You should see generations which are much less like training data! <sos> KING is now followed by words other than name of king names like before. Though the generations are much less clean now.

In [ ]:
random.seed(42)
np.random.seed(42)
bigram_lm = WordNGramLMWithAddKSmoothing(2, k=0.01)
bigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = bigram_lm.sample_text("<sos> KING", max_words=50)
    print(sampled_text)

Notice how with a smaller value of $k$, the generated sentences now start to resemble more with the training data

Test implementation of `eval_perplexity` for trigram, 4-gram, and 5-gram LMs with Laplace smoothing

In [ ]:
trigram_lm = WordNGramLMWithAddKSmoothing(3, k =1)
trigram_lm.fit(train_data_wth_unks)

train_ppl = trigram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = trigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model: {train_ppl}")
print(f"Dev Perplexity for Trigram model: {dev_ppl}")

print("\n\n")

fourgram_lm = WordNGramLMWithAddKSmoothing(4, k =1)
fourgram_lm.fit(train_data_wth_unks)
train_ppl = fourgram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = fourgram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 4-gram model: {train_ppl}")
print(f"Dev Perplexity for 4-gram model: {dev_ppl}")

print("\n\n")

fivegram_lm = WordNGramLMWithAddKSmoothing(5, k =1)
fivegram_lm.fit(train_data_wth_unks)
train_ppl = fivegram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = fivegram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 5-gram model: {train_ppl}")
print(f"Dev Perplexity for 5-gram model: {dev_ppl}")

You should roughly observe the following perplexities:

|N-gram LM | Train Perplexity | Dev Perplexity|
|----------|------------------|---------------|
| Trigram   | 1276              | 1829           |
| 4-gram  | 1710              | 3041         |
| 5-gram | 1794 | 3339 |


Test implementation of `sample_text` for trigram and 4-gram LMs with Laplace Smoothing

In [ ]:
def test_sample_text_ngram_laplace():
    print("Testing for Trigram model")

    random.seed(42)
    np.random.seed(42)
    trigram_lm = WordNGramLMWithAddKSmoothing(3)
    trigram_lm.fit(train_data)
    sampled_text = trigram_lm.sample_text("<sos> <sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos> <sos>")
    evaluate_test_case(
        None,
        sampled_text.startswith("<sos> <sos>"),
        True,
        output_str="Sampled text starts with <sos> <sos>",
    )

    print(
        "Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>"
    )
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 2}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 2 == 50 or (
        len(sampled_text.split()) - 2 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 3: Check if the probability of generating II is less than III when prefix is KING RICHARD"
    )
    sampled_texts = [
        trigram_lm.sample_text("KING RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("KING RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("KING RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)

    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 4: Check if the probability of generating II given KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "King Richard II",
        gen_prob_richard_2,
        0.0201,
        output_str="Probability of generating Richard II",
        atol=1e-2,
    )

    print(
        "Test Case 5: Check if the probability of generating III given KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "King Richard III",
        gen_prob_richard_3,
        0.0306,
        output_str="Probability of generating Richard III",
        atol=1e-2,
    )

    print("Testing for 4-gram model")
    fourgram_lm = WordNGramLMWithAddKSmoothing(4)
    fourgram_lm.fit(train_data)
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos>", max_words=50)

    print("Test Case 6: Check if the sampled text starts with <sos> <sos> <sos>")
    evaluate_test_case(
        None,
        sampled_text.startswith("<sos> <sos> <sos>"),
        True,
        output_str="Sampled text starts with <sos> <sos> <sos>",
    )

    print(
        "Test Case 7: Check if the number of generated words is either 50 or less than 50 and ends with <eos>"
    )
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 3}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 3 == 50 or (
        len(sampled_text.split()) - 3 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")

    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 8: Check if the probability of generating II is less than III when prefix is <sos> KING RICHARD"
    )
    sampled_texts = [
        fourgram_lm.sample_text("<sos> <sos> KING RICHARD", max_words=1)
        for _ in range(1000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "II"
        for text in sampled_texts
    ].count(True)

    num_richard_3s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "III"
        for text in sampled_texts
    ].count(True)

    gen_prob_rich2 = num_richard_2s / len(sampled_texts)
    gen_prob_rich3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_rich2}")
    print(f"Probability of generating Richard III: {gen_prob_rich3}")
    if gen_prob_rich2 < gen_prob_rich3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 9: Check if the probability of generating II given <sos> KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "<sos> King Richard II",
        gen_prob_rich2,
        0.0174,
        output_str="Probability of generating Richard II",
        atol=1e-2,
    )

    print(
        "Test Case 10: Check if the probability of generating III given <sos> KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "<sos> King Richard III",
        gen_prob_rich3,
        0.0295,
        output_str="Probability of generating Richard III",
        atol=1e-2,
    )
test_sample_text_ngram_laplace()

In [ ]:
np.random.seed(42)
random.seed(42)
print("Generations from Trigram model")
trigram_lm = WordNGramLMWithAddKSmoothing(3, k = 0.01)
trigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = trigram_lm.sample_text("<sos> <sos> KING", max_words=50)
    print(sampled_text)


In [ ]:
np.random.seed(42)
random.seed(42)
print("Generations from 4-gram model with Add-k smoothing")
fourgram_lm = WordNGramLMWithAddKSmoothing(4, k=0.01)
fourgram_lm.fit(train_data)
for _ in range(20):
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

In [ ]:
np.random.seed(42)
random.seed(42)
print("Generations from 5-gram model with Add-k smoothing")
fivegram_lm = WordNGramLMWithAddKSmoothing(5, k=0.01)
fivegram_lm.fit(train_data)
for _ in range(20):
    sampled_text = fivegram_lm.sample_text("<sos> <sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

In [ ]:
# </NO_AUTOGRADE>

### Write-Up Question 4 (1 Point)

Write expression for unigram, trigram, 4-gram, and 5-gram models with Laplace smoothing. (Your answer to this question should go in your separate write-up PDF.)

### Write-Up Question 5 (3 Points)

Plot how the train and dev perplexities of bigram, trigram, 4-gram, and 5-gram LMs with Add-k smoothing from different values of k -- $\{1e-8, 1e-7, \cdots, 1e-1, 1\}$. You should have perplexity on the y-axis and k on the x axis. Use log-scaling for the x axis when plotting. Explain the trend that you see in 3 lines. Finally, report the best setup i.e. values of $N$ and $k$ which achieve the best dev accuracy. (Your answer to this question should go in your separate write-up PDF; the following code block below is for the computation you'll need to do in preparation for your write-up.)

In [ ]:
# <NO_AUTOGRADE>

In [ ]:

# YOUR CODE HERE


In [ ]:
# </NO_AUTOGRADE>

### Exercise 3.3 Language Model Interpolation (10 Points)

An alternate to smoothing that often works well in practice is interpolating between different language models. Let's say we are trying to compute $P(w_n \mid w_{n-2} w_{n-1})$, but we have no examples of the particular trigram $w_{n-2}, w_{n-1} w_n$ in the training corpus, we can instead estimate its probability by using the bigram probability $P(w_n \mid w_{n-1})$. If there are no examples of the bigram $w_{n-1} w_n$ in the training data either, we use the unigram probability $P(w_n)$. Formally, the trigram probability by mixing the three distributions is given by:

$$\hat{P}(w_n \mid w_{n-2} w_{n-1}) = \lambda_1 P(w_n) + \lambda_2 P(w_n \mid w_{n-1}) + \lambda_3 P(w_n \mid w_{n-1} w_{n-2})$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$ (and each $\lambda$ is non-negative), making the above equation a form of weighted averaging. We can similarly write expressions for other N-gram LMs.

But how do we choose the values of different $\lambda_i$? We choose these values by tuning them on a held out data i.e. the dev set, very similar to tuning hyperparameters for a machine learning model.

In this exercise, you will implement the class `WordNGramLMWithInterpolation` similar to `WordNGramLM` and `WordNGramLMWithAddKSmoothing` that you did in the previous exercises but this time to support interpolation between different LMs.

In [ ]:
class WordNGramLMWithInterpolation(WordNGramLM):
    """
    Remember you can use the inheritance from WordNGramLM in your implementation!
    """

    def __init__(self, N: int, lambdas: List[float]):

        """
        Constructor for WordNGramLMWithInterpolation class.
        Inputs:
            - N: int, the N in N-gram
            - lambdas: List[float], the list of lambdas for interpolation between 1-gram, 2-gram, 3-gram, ..., N-gram models
                Note: The length of lambdas should be N. The sum of lambdas should be 1. lambdas[0] corresponds to 1-gram model, lambdas[1] corresponds to 2-gram model and so on.
        """


        raise NotImplementedError

    def fit(self, train_data: List[str]):

        """
        Trains an N-gram language model with interpolation.

        Inputs:
            - train_data: str, sentences in the training data

        """


        raise NotImplementedError

    def eval_perplexity(self, eval_data: List[str]) -> float:
        """
        Evaluates the perplexity of the N-gram language model with interpolation on the eval set.

        Input:
            - eval_data: List[str], the evaluation text

        Output:
            - float, the perplexity of the model on the evaluation set

        Note : For tokens that are not in the vocabulary, replace them with the <unk> token.

        """


        raise NotImplementedError

    def sample_text(self, prefix: str = "<sos>", max_words: int = 100) -> float:

        """
        Samples text from the N-gram language model with interpolation.

        Inputs:
            - prefix: str, the prefix to start the sampling from. Can also be multiple words separated by spaces.
            - max_words: int, the maximum number of words to sample

        Outputs:
            - str, the sampled text

        Note: Please use np.random.choice for sampling next words
        """


        raise NotImplementedError

    # Extra utility functions that you think will be useful can go below

    # YOUR CODE HERE


In [ ]:
# <NO_AUTOGRADE>

Test implementation of `eval_perplexity` for trigram model with Interpolation

In [ ]:
trigram_lm = WordNGramLMWithInterpolation(3, [0.3, 0.3, 0.4])
trigram_lm.fit(train_data_wth_unks)

train_ppl = trigram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = trigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model with Interpolation: {train_ppl}")
print(f"Dev Perplexity for Trigram model with Interpolation: {dev_ppl}")

You should see a train perplexity of around 13 and dev perplexity of 139. This should also be the best performing model based on dev perplexity that we have seen so far.

Test implementation of `eval_perplexity` for 4-gram model with Interpolation

In [ ]:
fourgram_lm = WordNGramLMWithInterpolation(4, [0.2, 0.4, 0.3, 0.1])
fourgram_lm.fit(train_data_wth_unks)

train_ppl = fourgram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = fourgram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model with Interpolation: {train_ppl}")
print(f"Dev Perplexity for Trigram model with Interpolation: {dev_ppl}")

Here you should get a train perplexity of roughly 8 and dev perplexity around 144.

Play around with different values of $\lambda_i$ and see how it effects the train and dev perplexities.

In [ ]:
np.random.seed(42)
random.seed(42)

print("Generations from Trigram model with Interpolation")
trigram_lm = WordNGramLMWithInterpolation(3, [0.3, 0.3, 0.4])
trigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = trigram_lm.sample_text("<sos> <sos> KING", max_words=50)
    print(sampled_text)

In [ ]:
np.random.seed(42)
random.seed(42)

print("Generations from 4-gram model with Interpolation")
fourgram_lm = WordNGramLMWithInterpolation(4, [0.2, 0.4, 0.3, 0.1])
fourgram_lm.fit(train_data)
for _ in range(20):
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

### Write-Up Question 6 (3 Points)

For bigram, trigram, and 4-gram models with interpolation, train models with different values of $\lambda_i$ and evaluate perplexities on train and dev datasets. You should try at least 5 sets of values for each model. Report the $\lambda_i$ values that you experiment with and train and dev perplexities for each setting and N-gram model. (Your answer to this question should go in your separate write-up PDF; the following code block below is for the computation you'll need to do in preparation for your write-up.)

In [ ]:

# YOUR CODE HERE


In [ ]:
# </NO_AUTOGRADE>